In [1]:
import cv2
import os
import h5py                                                                                                                                                                                   
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
%run Models.ipynb

## Define Generator

In [2]:
def Sequence_Data(path, batch_size):
    while True:
        # 宣告資料夾位置
        path_ori = os.path.join(path, "Original_pgm")
        path_dcmp = os.path.join(path, "Decompressed")
        
        # 取得資料 '檔案名稱'
        original = os.listdir(path_ori)
        decompressed = os.listdir(path_dcmp)
        
        # 檔案依照名稱排序，以利於同時在不同資料夾中取出相對應的資料
        original.sort()
        decompressed.sort()
        
        cnt = 0
        X =[]  
        Y =[]  
        for image in original:  
            # create Numpy arrays of input data
            # and labels, from each line in the file
            x = cv2.imread("{}/{}".format(path_dcmp, image),0)
            y = cv2.imread("{}/{}".format(path_ori, image),0)
            X.append(x)  
            Y.append(y)  
            
            cnt += 1
            if cnt==batch_size: 
                Xb = np.asarray(X)
                Yb = np.asarray(Y)
                cnt = 0
                yield (Xb.reshape(Xb.shape[0],64,64,1), Yb.reshape(Yb.shape[0],64,64,1))  
                X = []  
                Y = []  

## Training

In [ ]:
## 要先拿少量資料測試 ！！
batch_size = 1

# Step 1: 讀取訓練資料（受損 / 原圖）
path_train = "../../Dataset/Training_Data"  ## 受損影像路徑
path_valid = "../../Dataset/Validation_Data"  ## 原圖影像路徑
img_test = "/home/mj/HardDisk/Github/Image_Compressor/Dataset/Training_Data/Original_pgm/00006458.pgm"
img_test = cv2.imread(img_test, 0).reshape(1,64,64,1)
# Step 2: 讀取 Models
GN  = Get_GN_train()
# DN  = Get_DN_train()
# GDN = Get_GD_Stacked(GN, DN)

# Step 3: 初始化參數配置
optimizer = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, decay=8e-8)
GN.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
# DN.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# GDN.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


#
filepath="../../Model/weights-improvement-{epoch:03d}-{val_acc:.3f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,
mode='min')
callbacks_list = [checkpoint]



# Step : 先稍微訓練 Generative Network
for i in range(5):
    GN.fit_generator(generator=Sequence_Data(path_train, batch_size), steps_per_epoch=int(32000),
                        epochs=1, use_multiprocessing=True, #callbacks=[checkpoint], #workers=20
                        validation_data=Sequence_Data(path_valid, batch_size), validation_steps=int(4000))
    
    result = cv2.convertScaleAbs(GN.predict(img_test))
    print(result.shape)
    cv2.imshow("Img", result.reshape(64,64,1))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
#     cv2.imwrite("/home/mj/HardDisk/Github/Image_Compressor/Model/Result_{}.png".format(1), result)
    

# GDN.fit_generator(generator=Sequence_Data(path_train, batch_size), steps_per_epoch=int(16000),
#                     epochs=1, use_multiprocessing=True, #callbacks=[checkpoint], workers=20
#                     validation_data=Sequence_Data(path_valid, batch_size), validation_steps=int(4000))

# Step : 加入 Discriminiative Network 一起訓練， 儲存權重

# Step : 儲存 Final 權重

W0105 08:24:36.486299 140032793577216 deprecation.py:506] From /home/mj/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


 6917/32000 [=====>........................] - ETA: 12:56 - loss: 8651.1972 - acc: 4.0474e-04